In [1]:
#interactive map
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn import preprocessing


In [2]:
data=pd.read_csv('dataset1.csv')

In [3]:
#delete falcon1
data=data.drop(data[data['BoosterVersion']=='Falcon 1'].index)

In [3]:
#one hot for outcome
#先拆分字符串
data['Outcome_TN'], data['X'] = data['Outcome'].str.split(' ', 1).str
#Outcome_TN里面False和None是一样的，统一一下
data['Outcome_TN'] =data['Outcome_TN'].replace('False', 'None') 
#现在结果，对它做热独
df1=pd.get_dummies(data['Outcome_TN'], prefix='Outcome')  ## 其中 prefix 为新的特征名称前缀
#热独连接回原数据
DN=pd.concat([data, df1], axis=1)
#DN is the new data

<ipython-input-3-d8608737583e>:3: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  data['Outcome_TN'], data['X'] = data['Outcome'].str.split(' ', 1).str


In [4]:
import folium
import geopandas as gpd

from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

In [5]:
# define color in outcome
def outcome_color(val):
    if val == 1:
        return 'green'
    else:
        return 'red'
    
DN['outcome_color1'] = DN['Outcome_True'].apply(outcome_color)
DN.head(10)

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,...,Block,ReusedCount,Serial,Longitude,Latitude,Outcome_TN,X,Outcome_None,Outcome_True,outcome_color1
0,1,2006-03-24,Falcon 1,20.0,LEO,Kwajalein Atoll,None None,1,False,False,...,NaN,0,Merlin1A,167.743129,9.047721,None,None,1,0,red
1,2,2007-03-21,Falcon 1,NaN,LEO,Kwajalein Atoll,None None,1,False,False,...,NaN,0,Merlin2A,167.743129,9.047721,None,None,1,0,red
2,4,2008-09-28,Falcon 1,165.0,LEO,Kwajalein Atoll,None None,1,False,False,...,NaN,0,Merlin2C,167.743129,9.047721,None,None,1,0,red
3,5,2009-07-13,Falcon 1,200.0,LEO,Kwajalein Atoll,None None,1,False,False,...,NaN,0,Merlin3C,167.743129,9.047721,None,None,1,0,red
4,6,2010-06-04,Falcon 9,NaN,LEO,CCSFS SLC 40,None None,1,False,False,...,1.0,0,B0003,-80.577366,28.561857,None,None,1,0,red
5,8,2012-05-22,Falcon 9,525.0,LEO,CCSFS SLC 40,None None,1,False,False,...,1.0,0,B0005,-80.577366,28.561857,None,None,1,0,red
6,10,2013-03-01,Falcon 9,677.0,ISS,CCSFS SLC 40,None None,1,False,False,...,1.0,0,B0007,-80.577366,28.561857,None,None,1,0,red
7,11,2013-09-29,Falcon 9,500.0,PO,VAFB SLC 4E,False Ocean,1,False,False,...,1.0,0,B1003,-120.610829,34.632093,None,Ocean,1,0,red
8,12,2013-12-03,Falcon 9,3170.0,GTO,CCSFS SLC 40,None None,1,False,False,...,1.0,0,B1004,-80.577366,28.561857,None,None,1,0,red
9,13,2014-01-06,Falcon 9,3325.0,GTO,CCSFS SLC 40,None None,1,False,False,...,1.0,0,B1005,-80.577366,28.561857,None,None,1,0,red


In [6]:
m_11 = folium.Map(location=[28.6080585,-80.6039558], tiles='cartodbpositron', zoom_start=13)

# group
mc = MarkerCluster()
for idx, row in DN.iterrows():
    coordinate = [row['Latitude'], row['Longitude']]
    folium.map.Marker(coordinate, icon=folium.Icon(color='white',icon_color=row['outcome_color1'])).add_to(mc)
    
m_11.add_child(mc)

In [7]:
from folium.plugins import MousePosition

# Mouse swipe to display latitude and longitude
formatter = "function(num) {return L.Util.formatNum(num, 4) + ' º ';};"
plugin_hover = MousePosition(
    position='topright',
    separator=' | ',
    empty_string='Mouse swipe to display latitude and longitude',
    lng_first=False,
    num_digits=20,
    prefix='latitude and longitude:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

m_11.add_child(plugin_hover)

In [8]:
#add minimap
minimap = folium.plugins.MiniMap()
m_11.add_child(minimap)

In [9]:
# add measure
plugin_measure = folium.plugins.MeasureControl()
m_11.add_child(plugin_measure)

In [10]:
# change screen style by what you like
from branca.element import Figure

fig = Figure(width=750,height=450)
fig.add_child(m_11)

tile_list = ['openstreetmap', 'stamenwatercolor', 'cartodbdark_matter', 'cartodbpositron', 'cartodbpositronnolabels', 'cartodbpositrononlylabels', 'stamentonerlabels', 'stamenterrain', 'stamentoner', 'stamentonerbackground']

for each in tile_list:
    folium.TileLayer(each).add_to(m_11)

folium.LayerControl().add_to(m_11)

In [11]:
m_11

In [14]:
m_11.save("maps11.html")